In [ ]:
!pip install -qU langchain langchain-openai langchain-community langchain-experimental

# Build a Simple LLM Application with LCEL

In this session, we will build a simple LLM application with LangChain. This application will translate text from English into another language.

## Setup

Many of the applications we build with LangChain will contain multiple steps with multiple invocations of LLM calls. It becomes crucial to inspect what exactly is going on inside our chain or agent as these applications gets more and more complex. It is better to do this with LangSmith.

In [ ]:
import os

langchain_api_key = 'your_langchain_api_key_here'  # Replace with your actual LangChain API key
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_API_KEY'] = langchain_api_key

## Using Language Models

Use OpenAI as a n example. (Other models can be found in the official tutorial pages.)

In [ ]:
openai_api_key = 'your_openai_api_key_here'  # Replace with your actual OpenAI API key
os.environ['OPENAI_API_KEY'] = openai_api_key

In [4]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model='gpt-3.5-turbo')

`ChatModel`s are instances of LangChain "Runnables", which means they expose a standard interface for interacting with them. To call the model, we can pass in a list of messages to the `.invoke()` method.

In [5]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into French."),
    HumanMessage(content="I love programming.")
]

result = model.invoke(messages)
result

AIMessage(content="J'adore la programmation.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 23, 'total_tokens': 31, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2bfa78c3-34f3-4e57-ab7d-3ea32058d480-0', usage_metadata={'input_tokens': 23, 'output_tokens': 8, 'total_tokens': 31, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

## OutputParsers

The `AIMessage` object contains a string response along with other metadata about the response. We can parase out this response by using a simple output parser:

In [6]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [7]:
parser.invoke(result)

"J'adore la programmation."

We can "chain" the model with this output parser. This chain takes on the input type of the language model (string or list of message) and returns the output type of the output parser (string).

We can create the chain using `|` operator.

In [8]:
chain = model | parser

In [9]:
chain.invoke(messages)

"J'adore programmer."

## Prompt Templates

We passed a list of messages (`HumanMessage` and `SystemMessag`) into the language model. It is constructed from a combination of user input and application logic. This application logic usually takes the raw user input and transforms it into a list of messages ready to pass to the language model.

`PromptTemplates` are a concept in LangChain designed to assist with this transformation. They take in raw user input and return data (a prompt) that is ready to pass into a language model.

In [11]:
from langchain_core.prompts import ChatPromptTemplate

In [13]:
system_template = 'Translate the following into {language}'

prompt_template = ChatPromptTemplate.from_messages(
    [
        ('system', system_template),
        ('user', '{text}')
    ]
)

This PromptTemplate is a combination of the `system_template` as well as a simpler tempalte for where to put the text to be translated.

The input to this prompt template is a dictionary. We can play around with this prompt template by itself to see what it does by itself:

In [15]:
result = prompt_template.invoke(
    {'language': 'french',
     'text': 'I love programming.'}
)
result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into french', additional_kwargs={}, response_metadata={}), HumanMessage(content='I love programming.', additional_kwargs={}, response_metadata={})])

If we want to access the messages directly:

In [16]:
result.to_messages()

[SystemMessage(content='Translate the following into french', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I love programming.', additional_kwargs={}, response_metadata={})]

## Chaining together components with LCEL

We can combine the PromptTemplate with the model and the output parser using the pipe operator:

In [17]:
chain = prompt_template | model | parser

In [18]:
chain.invoke(
    {'language': 'french',
     'text': 'I love programming.'}
)

"J'adore programmer."

This is a simple example of using **LangChain Expression Language (LCEL)** to chain together LangChain modules.

## Serving with LangServe

LangServe helps developers deploy LangChain chains as a REST API. To install,
```bash
pip install "langserve[all]"
```

### Server

To create a server for our application, we need to make a `serve.py` file, which contains our logic for serving our application. It consists of three things:
1. the definition of our chain
2. our FastAPI app
3. a definition of a route from which to serve the chain

```python
# This is what serve.py should have.

#!/usr/bin/env python
from fastapi import FastAPI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langserve import add_routes

# 1. Create prompt template
system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages([
    ('system', system_template),
    ('user', '{text}'),
])

# 2. Create model
model = ChatOpenAI()

# 3. Create parser
parser = StrOutputParser()

# 4. Create chain
chain = prompt_template | model | parser

# 5. App definition
app = FastAPI(
    title='LangChain Server',
    version='1.0',
    description="A simple API server using LangChain's Runnable interfaces",
)

# 6. Add chain route
add_routes(
    app,
    chain,
    path='/chain',
)

if __name__ == '__main__':
    import uvicorn

    uvicorn.run(app, host='localhost', port=8080)

```

If we execute this file:
```bash
python serve.py
```
we should see our chain being served at http://localhost:8080.

### Playground

Every LangServe service comes with a simple built-in UI for configuring and invoking the application with streaming output and visibility into intermediate steps.

If we visit http://localhost:8080/chain/playground and pass `{'language': 'french', 'text': 'I love programming.'}`, we should get the same result.

### Client

Now that the service is online, we can set up a client for programmatically interacting with our service using `langserve.RemoteRunnable`. Using this, we can interact with the served chain as if it were running client-side:
```python
from langserve import RemoteRunnable

remote_chain = RemoteRunnable('http://localhost:8080/chain/')
remote_chain.invoke(
    {'language': 'french',
     'text': 'I love programming.'}
)
```